<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Multiple_Times_modeling_using_Apache_Spark_and_Facebook_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"]= "usr/lib/jvm/java-8-jdk-amd64/jre/bin/java"
!update-alternatives --set java /usr/lib/jvm/java-8-jdk-amd64/jre/bin/java

update-alternatives: error: alternative /usr/lib/jvm/java-8-jdk-amd64/jre/bin/java for java not registered; not setting


In [2]:
!java --version

openjdk 11.0.8 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [3]:
!pip install pyspark

In [4]:
!pip install pyarrow==0.15.1

In [5]:
!pip install fbprophet

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from fbprophet import Prophet

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

In [7]:
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.master('local').getOrCreate()

Exception: ignored

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/weekly_sales_data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df['date']= pd.to_datetime(df['date'],infer_datetime_format=True)

In [ ]:
df.info()

In [ ]:
print("\nFeatures         :\n",df.columns.to_list())
print("\Missing Values    :\n",df.isnull().any())
print("\nUnique Values    :\n", df.nunique())

In [ ]:
df.describe()

In [ ]:
df_base= df.copy()

In [ ]:
df['store_id'].value_counts()

In [ ]:
item_df= df.set_index('date')

In [ ]:
item_df.query("store_id==41")[['sales']].plot();

In [ ]:
item_df.query("store_id==25")[['sales']].plot();

In [ ]:
item_df.query("store_id==29")[['sales']].plot();

In [ ]:
sdf=spark.createDataFrame(df)

In [ ]:
sdf.show(5)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.count()

In [ ]:
sdf.createOrReplaceTempView("sales")

In [ ]:
spark.sql("select store_id,count(*) from sales group by store_id order by store_id").show()

In [ ]:
sql = 'SELECT store_id, date as ds, sum(sales) as y FROM  sales GROUP BY store_id, ds ORDER BY store_id, ds'

In [ ]:
spark.sql(sql).show()

In [ ]:
sdf.explain()

In [ ]:
sdf.rdd.getNumPartitions()

In [ ]:
store_part=(spark.sql(sql).repartition(spark.sparkContext.defaultParallelism,['store_id'])).cache()

In [ ]:
store_part

In [ ]:
store_part.explain()

In [ ]:
from pyspark.sql.types import *

result_schema =StructType([
                           StructField('ds',TimestampType()),
                           StructField('store_d',IntegerType()),
                           StructField('y',DoubleType()),
                           StructField('yhat',DoubleType()),
                           StructField('yhat_upper',DoubleType()),
                           StructField('yhat_lower',DoubleType())
])

In [ ]:
from pyspark.sql.functions import pandas_udf,PandasUDFType

@pandas_udf(result_schema,PandasUDFType.GROUPED_MAP)

def forecast_sales(store_pd):
  model =Prophet(interval_width=0.95,seasonality_mode='multiplicative',weekly_seasonality=True,yearly_seasonality=True)
  model.fit(store_pd)

  future_pd = model.make_future_dataframe(
      periods=5,
      freq='w'
  )
  forecast_pd = model.predict(future_pd)

  f_pd =forecase_pd[['ds','yhat','yhat_upper','yhat_lower']].set_index('ds')
  st_pd = store_pd[['ds','store_id','y']].set_index('ds')

  result_pd =f_pd.join(st_pd,how='left')
  result_pd.reset_index(level=0, inplace=True)
  result_pd['store_pd']= store_pd['store_pd'].iloc[0]

  return result_pd [['ds','store_id','y','yhat','yhat_upper','yhat_lower']]



In [ ]:
from pyspark.sql.functions import current_date

results=(
    store_part
    .groupBy('store_id')
    .apply(forecast_sales)
    .withColumn('training_date',current_date())

)

In [ ]:
results.cache()

In [ ]:
results.show()

In [ ]:
results.explain()

In [ ]:
results.coalesce(1)

In [ ]:
results.createOrReplaceTempView('forcasted')

In [ ]:
spark.sql("select store_id,count(*) from forcasted groupby store_id").show()

In [ ]:
final_df= results.toPandas()

In [ ]:
final_df

In [ ]:
final_df.reset_index('ds')

In [ ]:
final_df.query('store_id == 44')[['y','yhat']].plot()